# Metacluster Remapping GUI

- Usage
- The tool itself
- Start options
- File formats
- Developer guid

## Usage

### Quickstart
- **Select**: Left Click
- **Remap**: Right Click
- **Edit Metacluster Name**: Textbox below the heatmaps

All changes are saved immediately to the output file.

### Selection and Remapping details
- To select one or more clusters, click anywhere on either heatmap. Clicking a selected cluster will deselect it.
- To remap the selected clusters, right click anywhere on either heatmap.

### Other features
- The bars at the top show the relative size of each cluster.
- If you remap all cluster in a metacluster, the metacluster is now *gone*
- You can click the **new metacluster** button to add `current selection` to a new metacluster
- Adjust the zscore limit using the slider to adjust your dynamic range.



In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
plt.ion()

import sys
from ark.utils.metacluster_remap_gui import MetaClusterData, MetaClusterGui, metaclusterdata_from_files
mcd = metaclusterdata_from_files(
    "../data/example_dataset/metaclustering/ex1_clusters_nozscore.csv",
    "../data/example_dataset/metaclustering/ex1_clusters_pixelcount.csv", metacluster_header='hCluster_cap')
mcd.output_mapping_filename = "../data/example_dataset/metaclustering/ex1_output_mapping.csv"
mcg = MetaClusterGui(mcd, width=17, debug=False)

## Startup Options

In [ ]:
metaclusterdata_from_files?

In [ ]:
MetaClusterGui?

## File Formats
### Input Format
clusters_nozscore.csv
```
CD14,CD209,...,cluster,hCluster_cap
2.82664769656717e-05,2.36648137078826e-05,...,5,1
0.000200020182840592,0.000130916281881885,...,88,1
...
0.000200020182840592,0.000130916281881885,...,32,20
```
pixelcount.csv
```
cluster,count
1,351025
2,51388
3,21746
...
100,58730

```
Note: the metacluster columnname is configurable, see above.

### Output Format
```
cluster,metacluster,mc_name
1,10,CD4 T Cells
2,10,CD4 T Cells
3,9,9
...
100,13,13
```


Set your output file using:
```
mcd.output_mapping_filename = "ex1_output_mapping.csv"
```

# Developers Maintanence Guide

## High level Architecture

- `MetaClusterData` - Data IO and state management
- `MetaClusterGui` - Handle the user interactivty

This metacluster remapper GUI uses the ipywidgets backend for Matplotlib. Addition widgets are used for extra input/output.

## Running unit tests
From the root of the ark-analysis repo run:

    pytest': python -m pytest --cov=ark/utils/metacluster_remap_gui/ --pycodestyle ark/utils/metacluster_remap_gui/
    
Tests are located in the same folder as the non-test modules.

## How to debug callbacks
All of the callbacks are already connected to a debug output region. Enable this with the `MetaClusterGui(..., debug=True)`.
https://ipywidgets.readthedocs.io/en/latest/examples/Output%20Widget.html#Debugging-errors-in-callbacks-with-the-output-widget

## Proof of concept for adding hover annotations

Requires upgrading matplotlib 3.3.4 from 2.2.5 (2021-09-13). Also it is kinda laggy and jumpy, but does work. 

    import mplcursors
    mplcursors.Cursor
    a = None
    def annote(sel):
        global a
        a = sel
        cluster = mcd.clusters.index[int(sel.target[0])]
        ann = f"Cluster: {cluster}\n"
        ann += f"Meta: {mcd.get_metacluster_displayname(mcd.which_metacluster(cluster))}"
        return sel.annotation.set_text(ann)


    mplcursors.cursor(mcg.im_c, hover=True).connect("add", annote)